## 任务：京东用户购买意向预测

### 故事背景：
京东作为中国最大的自营式电商，在保持高速发展的同时，沉淀了数亿的忠实用户，积累了海量的真实数据。如何从历史数据中找出规律，去预测用户未来的购买需求，让最合适的商品遇见最需要的人，是大数据应用在精准营销中的关键问题，也是所有电商平台在做智能化升级时所需要的核心技术。

以京东商城真实的用户、商品和行为数据（脱敏后）为基础，通过数据挖掘的技术和机器学习的算法，构建用户购买商品的预测模型，输出高潜用户和目标商品的匹配结果，为精准营销提供高质量的目标群体。

目标：使用京东多个品类下商品的历史销售数据，构建算法模型，预测用户在未来5天内，对某个目标品类下商品的购买意向。

### 数据集：
这里涉及到的数据集是京东的数据集:

* JData_User.csv 用户数据集 105,321个用户
* JData_Comment.csv 商品评论 558,552条记录
* JData_Product.csv 预测商品集合 24,187条记录
* JData_Action_201602.csv 2月份行为交互记录 11,485,424条记录
* JData_Action_201603.csv 3月份行为交互记录 25,916,378条记录
* JData_Action_201604.csv 4月份行为交互记录 13,199,934条记录

**JData_User.csv用户数据**

|字段|意义|备注|
|-|-|-|
|user_id|用户id|脱敏|
|age|年龄|-1表未知|
|sex|性别|0男，1女，2未知|
|user_lv_cd|用户等级|级别枚举，越高级别越大|
|user_reg_tm|用户注册日期|粒度到天|

**JData_Comment.csv评论数据**

|字段|意义|备注|
|-|-|-|
|dt|截止时间|天，到2016-02-01|
|sku_id|商品编号|脱敏|
|comment_num|累积评论数分段|0表示无评论，1表是1条，2表示2-10条，3表示11-50条，5表示大于50条|
|has_bad_comment|是否有差评|0表示无，1表示有|
|bad_comment_rate|差评率|差评数占总评论数的比率|

**JData_Product.csv商品数据**

|字段|意义|备注|
|-|-|-|
|sku_id|商品编号|脱敏|
|a1|属性1|枚举，-1表未知|
|a2|属性2|枚举，-1表未知|
|a3|属性3|枚举，-1表未知|
|cate|品牌ID|脱敏|
|brand|品牌ID|脱敏|

**JData_Action_xx.csv商品数据**

|字段|意义|备注|
|-|-|-|
|user_id|用户ID|脱敏|
|sku_id|商品编号|脱敏|
|time|行为时间||
|model_id|点击板块的编号|脱敏|
|type|行为类型|1.浏览商品详情页;2.加入购物车;3.购物车删除;4.下单;5.关注;6.点击;|
|cate|品牌ID|脱敏|
|brand|品牌ID|脱敏|